
__rel / nonRel 이란?__

rel / nonRel  * TF * IDF 

후드티 스웨터 드레스 
바지 신발 주방용품

* 분석 대상인 카테고리의 문서들과, 분석 대상이 아닌 카테고리의 문서들을 모두 가지고 있을 경우, 특정단어가 분석 대상인 카테고리에 해당하는 단어인지를 수치로 표현 하기 위한 방법.
* (근본없는 방법입니다.. 한번 시도해보고 별로면 팽)

* __등장배경__
    * IDF가 stopWords들을 잘 걸러주지만, 유의미한 단어들(옷의 특징이 되는 단어들이지만 많은 옷에 등장해서 idf값이 낮아진)또한 걸러낸다는 문제점을 보안하기 위함.
    
* __단점__
    * 쓸데없는 데이터까지 크롤링해야함.. 그만큼 연산도 오래걸림..bow 길이 늘어남.. 

__현재 상황__

* 일단은 필요없어 보이는 카테고리(치마,바지)의 데이터도 저장해둠.
* 분석대상은 스웨터, 드레스(+ 후디)
* 즉, 어떤 단어가 스웨터, 드레스(+후디)를 나타내는 단어인지 알아야 한다.
-----
1.  rel Category : 스웨터, 드레스
2.  nonRel Category : 치마, 바지
------
__방법__

1. 전체 카테고리(토픽)을 두 부류로 나눔 => (이 경우엔)옷과관련된 카테고리, 아닌 카테고리

2. 모든 단어에 rel / nonRel 값을 부여
    1. rel : 관련 카테고리에 등장한 횟수 
    2. nonRel : 관련x 카테고리에서 등장한 횟수
3. rel / nonRel : 
    * 이 값이 클수록? => 우리가 원하는 주제와 관련된 단어이다
    
    ### B01AA6PPXU 얘 버려
    
    ### dict(  [(  ,  ),(  ,  ),(  ,  )]  )
    
   
# * 사용할 라이브러리 + 함수

In [1]:
import json
from pathlib import Path  # Path클래스 임포트
import re  # 특수문자 제거
from functools import cmp_to_key
import boto3
from nltk.tokenize import RegexpTokenizer

#내림차순 정렬을 위한 함수객체들
def cmp(l,r):
    if l>r:
        return 1
    elif l==r :
        return 0
    else:
        return -1
def key_func(l,r):
    if l[1]==r[1]:
        return cmp(l[0],r[0])
    return -cmp(l[1],r[1])

#dictionary => 내림차순 정렬된 list of tuple으로 만들어서 리턴
def sortDict(bow):
    word_list=[] #리스트에 (단어, 횟수) 꼴의 튜플을 넣을 것

    for i,k in bow.items() :
        word_list.append((i,k))
    word_list.sort(key=lambda x:x[1], reverse=True) #횟수를 기준으로 내림차순 정렬
    word_list.sort(key=cmp_to_key(key_func))# 횟수는같을때, 단어를 기준으로 정렬
    return word_list
#list of tuple을 출력
def displayTupList(tupList, num, withVal=False):  # 디폴트는 내림차순, 값 없이..
    counter = 0
    for tup in tupList:
        if tup[1] == 0 or counter == num:
            break
        else:
            if withVal:
                print(tup[0], " : ", tup[1], "  ", end="")
            else:
                print(tup[0], "   ", end="")
            counter += 1
    print()
#어떤 특수기호를 제거할지?
def delSymbol(feature):
    str = re.sub('[^A-Za-z0-9-]+', ' ', feature)  # 알파벳, '-'  빼고 다 지움. # str = re.sub('[^A-Za-z0-9-%]+', ' ', feature)
    str = re.sub('[0-9]+', 'ㅠ', str)  # 숫자를 ㅠ 로 바꿈 (나중에 덩어리째로 지우기)
    while (str.find("--") != -1):  # - 가 두개이상 있는경우엔 지움
        str = str.replace("--", " ")
    return str.strip('-')  # -가 사이에 낀게 아니라 겉에남아있는경우엔 제거

#버릴단어인지?
# 일단은 -- , % 만 남거나, 숫자만 남으면 버림
def isSemantic(word):
    if word == '' or word == '-' or word == '%' or word.isdigit() or word.find("ㅠ") != -1:
        return False
    else:
        return True

#경로에 있는 모든 id.json 파일의 features로 bag of word(딕셔너리)를 만들어 리턴
def makeBow(path) :
    for item in path.iterdir():
        if(item.is_dir()): #안보이는 디렉토리가 있다.. 주의
            continue
        itemJson = json.loads(item.read_text(), encoding='utf-8')
        features = itemJson["features"]

        for feature in features:
            feature = delSymbol(feature)  # 1. 특수기호, 이모티콘을 띄어쓰기로 대체
            for word in feature.split():  # 2. 띄어쓰기 단위로 분리
                word = word.lower().strip('-')  # 3. 겉에 남은 -  제거!
                if not isSemantic(word):  # 4. 무의미한 word는 버림
                    continue
                if word in bow:
                    bow[word] += 1
                else:
                    bow[word] = 1
    return bow
#특정 문서(features)하나 만의 bag of word를 만들고, 단어개수 리턴.
# 아니 근데 등장하는 단어만 있어도 되는거 아니냐?.... 내가 오ㅐ 이랬지?
# 계산은 그리 오래 걸리지 않는데, json으로 저장할 때 좀 심각해.. 
def makeSmallBow(features) :
    smallBow = {}
    for word, count in bow.items():
            smallBow[word] = 0
    counter = 0 #단어개수
    for feature in features:
        feature = delSymbol(feature) # 1. 특수기호, 이모티콘을 띄어쓰기로 대체
        for word in feature.split(): #2. 띄어쓰기 단위로 분리
            word = word.lower().strip('-')  #3. 겉에 남은 -  제거!
            if not isSemantic(word): # 4. 무의미한 word는 버림
                continue
            smallBow[word] += 1
            #print(word, "카운트 1 증가")
            counter += 1
    return smallBow, counter

def tokenize(feature):
    tokens = []
    feature = delSymbol(feature)
    for token in feature.split():
        token = token.lower().strip('-')
        if isSemantic(token):
            tokens.append(token)
    return tokens

#문서 하나의 tf를 계산 (dictionary)                
def computeTF(bow, features):  
    smallBow, numOfWord = makeSmallBow(features)  # eachBow : 특정 상품의 word count dict
    # numOfWord : 특정 상품의 word 개수
    tf = {}  # 단어별 tf를 계산해서 넣음
    for word, count in smallBow.items():
        if (numOfWord == 0):  # feature 없는 상품..
            break
        tf[word] = count / float(numOfWord)
    return tf

#모든 tf  가지고, idf 를 계산 (dictionary)
def computeIDF(tfs) :
    import math
    idf = {}
    for word, count in bow.items():
            idf[word] = 0
    N = len(tfs)  # 모든 상품 개수
    for ID, tf in tfs.items(): 
        for word, val in tf.items():
            if val > 0:
                idf[word] += 1  # 일단은 "word : word를 가진 상품개수" 꼴의 사전 완성..
    for word, val in idf.items():
        idf[word] = math.log10(N / float(val))
    return idf

#문서 하나의 tf-idf 계산 (dictionary)
#(시도1) : IDF / TF 
# 다른문서에서 자주 등장해야함. 
# 이 문서에서는 적게 등장해야함.  (애초에 중요한 단어는 한번만 등장할테니) => 그런 의미에서 TF = 0 이면 버림
def computeIDFdivTF(tfBow, idfs):
    idftf = {} 
    for word, val in tfBow.items():
        if val == 0:
            idftf[word] = 0
        else:
            idftf[word] = idfs[word] / val
    return idftf
def computeTFIDF(tfBow, idfs): 
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val * idfs[word]
    return tfidf

#path 에 있는 모든 id.json 파일을 가지고, tf를 구하여 "ID" : tf   꼴의 딕셔너리 리턴
def computeAllTF(path):
    count = 0
    for item in path.iterdir():
        if(item.is_dir()): #안보이는 디렉토리가 있다.. 주의 ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
            continue
        itemJson = json.loads(item.read_text(), encoding='utf-8')
        features = itemJson["features"]
        ID = itemJson["id"]
        
        tf = computeTF(bow, features)
        tfDict[ID] = tf
    return tfDict

# 0. rel / nonRel 계산 

1. 아니.. non relative에서 한번도 안나온 단어면..? 중요한단어일수도있고 오타일수도 있고 -ㅡㅡ 으음..
    * 최댓값으로 설정? (586)
2. 값의 범위가 너무 넓음 ㅡㅡ : 0 ~ 586.. 
    * (rCount/rTotalWordCount) * (nCount/nTotalWordCount)

In [2]:
path1 = Path('/home/jovyan/work/크롤링/relFeatures')
path2 = Path('/home/jovyan/work/크롤링/nonRelFeatures')
paths = [path1, path2]

# 1. Path1, path2 를 통틀어 하나의 BOW 를 만든다.
# 2. 동시에 relNonRel Dict 역시 만든다.  word : {rel: , nonRel: }
def bowAndRel(paths) :
    bow = {}
    relNonRelDict = {}  # word : {"R": , "N": }
    for path in paths:
        for item in path.iterdir():
            if(item.is_dir()): 
                continue
            itemJson = json.loads(item.read_text(), encoding='utf-8')
            features = itemJson["features"]
            category = itemJson["category"]
            RN = None
            if category == "sweater" or category == "hoodies&sweatshirts" or category == "dress":
                RN = "R"
            elif category == "skirt" or "pants" or "nonRel":
                RN = "N"
            else:
                print("에러!")
            featureStr = ""
            for feature in features:
                featureStr = featureStr + " " +  feature + " "
            tokens = tokenize(featureStr)
            for token in tokens:
                if token in bow:
                    bow[token] += 1
                else:
                    bow[token] = 1
                if token not in relNonRelDict:
                    relNonRelDict[token] = {"R":0, "N":0}
                relNonRelDict[token][RN] += 1
                    
    return bow, relNonRelDict
bow, relNonRelDict = bowAndRel(paths)

rnrDict = {}
counter = 0
for token, val in relNonRelDict.items():
    if val["N"] == 0:
        counter += 1
        rnrDict[token] = 999
    else:
        rnrDict[token] = val["R"] / val["N"] 
        #아니.. non relative에서 한번도 안나온 단어면..? 중요한단어일수도있고 오타일수도 있고 -ㅡㅡ 으음..
        # 1. 최댓값으로 설정? (586)
        # 2. 값의 범위가 너무 넓음 ㅡㅡ : 0 ~ 586.. 

In [27]:
print(counter, " / ", len(rnrDict))  # 딱 절반의 단어가 non-rel 에는 등장하지 않음.. 

8209  /  17509


In [5]:
json.dump(rnrDict, open(f'/home/jovyan/work/분석/relNonRel' + '.json', 'w+')) 

In [25]:
displayTupList(sortDict(rnrDict), -1, withVal = True)
# 1. idf가 낮을것 (여기저기 잘 등장할것) + r/n 값이 클것 (상의에만 잘 등장할 단어일것)
# 2. 고민) NonRel = 0 인경우.. rel에만 등장한 단어인데.. (거의 절반임) 
# =======>  주로, 의미없는단어, 오타 등등인 경우가 많음..

a-grade  :  999   a-lien  :  999   a-little-long  :  999   a-z  :  999   aamzon  :  999   ababalaya  :  999   abaya  :  999   abayas  :  999   abberation  :  999   abbreviation  :  999   abd  :  999   abes  :  999   ablaze  :  999   abluent  :  999   abollria  :  999   above-knees  :  999   above-the-ankle  :  999   absoluely  :  999   absorbant  :  999   absorbency  :  999   absord  :  999   absortion  :  999   abundant  :  999   abvoe  :  999   abyssian  :  999   ac  :  999   academic  :  999   accasion  :  999   accassion  :  999   accation  :  999   accations  :  999   accebtuate  :  999   acceessories  :  999   accentuating  :  999   acceptable  :  999   accepted  :  999   access-pull  :  999   accessibility  :  999   accessorized  :  999   accessorizes  :  999   accessorizing  :  999   accident  :  999   accodring  :  999   accoridng  :  999   accpet  :  999   accurately  :  999   accylic  :  999   acharline  :  999   ackets  :  999   acomfortable  :  999   acryli  :  999   acryl

bead-work  :  999   beadings  :  999   beadwork  :  999   bean  :  999   beard  :  999   bearing  :  999   bears  :  999   beast  :  999   beat  :  999   beaten  :  999   beatifully  :  999   beause  :  999   beaut  :  999   beautidul  :  999   beauties  :  999   beautife  :  999   beautifull  :  999   beautifully-fitting  :  999   beautifully-made  :  999   beautify  :  999   beautigul  :  999   bebonnie  :  999   becase  :  999   becuase  :  999   bedazzled  :  999   bedminton  :  999   bedroom  :  999   bedrooms  :  999   bedtime  :  999   bedwear  :  999   bee  :  999   beechwood  :  999   beer  :  999   beets  :  999   befitting  :  999   befort  :  999   befroe  :  999   beg  :  999   begin  :  999   begonia  :  999   behaviors  :  999   behinds  :  999   beja  :  999   belego  :  999   belend  :  999   belended  :  999   belief  :  999   believin  :  999   bell-accented  :  999   bell-like  :  999   bell-shaped  :  999   bell-sleeve  :  999   bell-sleeves  :  999   belle  :  999

cannnot  :  999   cansual  :  999   cantact  :  999   cap-sleeve  :  999   cap-sleeves  :  999   capabilities  :  999   capability  :  999   capcom  :  999   cape-back  :  999   cape-style  :  999   capelet  :  999   capes  :  999   caplet  :  999   cappa  :  999   capped  :  999   caprics  :  999   capsule  :  999   captain  :  999   captivate  :  999   captivates  :  999   captured  :  999   capturing  :  999   car  :  999   caradigan  :  999   carbon-black  :  999   carbon-fibre  :  999   carded  :  999   cardgian  :  999   cardgigan  :  999   cardi  :  999   cardiagn  :  999   cardiga  :  999   cardigan-stitch  :  999   cardigan-sweaters  :  999   cardiganis  :  999   cardigant  :  999   cardigian  :  999   cardign  :  999   cardigon  :  999   cardinal  :  999   cardis  :  999   cared  :  999   careen  :  999   cargigan  :  999   carhartt  :  999   caridan  :  999   caridgan  :  999   caridigan  :  999   cariety  :  999   carings  :  999   carolina  :  999   caroling  :  999   carp

999   complementing  :  999   completed  :  999   completer  :  999   complex  :  999   complimennts  :  999   complimentary  :  999   complimented  :  999   comply  :  999   comport  :  999   composite  :  999   comprised  :  999   compromised  :  999   comy  :  999   conact  :  999   conbimation  :  999   conc  :  999   concealer  :  999   concealing  :  999   concentrates  :  999   conceptation  :  999   conception  :  999   conditioned  :  999   conditioner  :  999   conditioning  :  999   conditionour  :  999   condole  :  999   coner  :  999   confidante  :  999   confirmation  :  999   confirmed  :  999   confirming  :  999   conformable  :  999   confront  :  999   confuse  :  999   confusion  :  999   conjoined  :  999   connected  :  999   connecting  :  999   connection  :  999   conquers  :  999   conscience  :  999   conserve  :  999   considerable  :  999   consideration  :  999   conspicuous  :  999   conspire  :  999   conspires  :  999   constant  :  999   constituents

 999   desgign  :  999   desian  :  999   desigan  :  999   desigh  :  999   design-hooded  :  999   design-it  :  999   designated  :  999   designation  :  999   designful  :  999   desined  :  999   desiner  :  999   desing  :  999   desings  :  999   desins  :  999   desirepath  :  999   desitination  :  999   despicable  :  999   desprition  :  999   desription  :  999   dess  :  999   desses  :  999   destined  :  999   destoyred  :  999   destroy  :  999   desxription  :  999   detached  :  999   detai  :  999   detailes  :  999   detect  :  999   detergents  :  999   deting  :  999   detract  :  999   devation  :  999   development  :  999   deviate  :  999   deviations  :  999   devil  :  999   devoicingits  :  999   devote  :  999   dew  :  999   dh  :  999   dialogue  :  999   dialy  :  999   diamond-quilted  :  999   diamond-type  :  999   dibble  :  999   dickey  :  999   dickeys  :  999   didnt  :  999   die  :  999   diesel  :  999   differenr  :  999   different-floral 

...

 :  1.5   varieties  :  1.5   watercolor  :  1.5   yarn-dyed  :  1.5   zips  :  1.5   crop  :  1.4952830188679245   added  :  1.4838709677419355   performance  :  1.48   feather  :  1.4782608695652173   shapes  :  1.4772727272727273   they  :  1.471111111111111   adjust  :  1.4705882352941178   tech  :  1.4666666666666666   binding  :  1.4615384615384615   brands  :  1.4615384615384615   fixed  :  1.4615384615384615   management  :  1.4615384615384615   lower  :  1.4594594594594594   ok  :  1.4594594594594594   exciting  :  1.4545454545454546   standards  :  1.4545454545454546   dyed  :  1.4528301886792452   prints  :  1.4477611940298507   diagonal  :  1.4444444444444444   fish  :  1.4444444444444444   handy  :  1.4444444444444444   johnny  :  1.4444444444444444   models  :  1.4444444444444444   shorter  :  1.4444444444444444   prevent  :  1.4426229508196722   zip  :  1.4424131627056673   assured  :  1.44   narrow  :  1.4375   slash  :  1.4352941176470588   drawstring  :  1.43044189852

  :  1.0   academy  :  1.0   accessible  :  1.0   accroding  :  1.0   adjustbale  :  1.0   adjusts  :  1.0   afibi  :  1.0   agan  :  1.0   air-dry  :  1.0   alexa  :  1.0   all-cotton  :  1.0   alongside  :  1.0   amber  :  1.0   ample  :  1.0   annflat  :  1.0   anniversaries  :  1.0   antimicrobial  :  1.0   anymore  :  1.0   apparent  :  1.0   apperal  :  1.0   applications  :  1.0   appropriately  :  1.0   appx  :  1.0   armor  :  1.0   aroud  :  1.0   arrives  :  1.0   artisanal  :  1.0   assistant  :  1.0   assures  :  1.0   assymetric  :  1.0   attachment  :  1.0   au  :  1.0   august  :  1.0   aundry  :  1.0   avaliable  :  1.0   baisc  :  1.0   balanced  :  1.0   bangles  :  1.0   bbyes  :  1.0   becoming  :  1.0   belongings  :  1.0   bends  :  1.0   bird  :  1.0   blossom  :  1.0   blowing  :  1.0   boardroom  :  1.0   bodies  :  1.0   body-shape  :  1.0   bodyshaper  :  1.0   bollywood  :  1.0   bombax  :  1.0   bonus  :  1.0   boring  :  1.0   bota  :  1.0   botanical  : 

buckles  :  0.8   calm  :  0.8   canvas  :  0.8   ellen  :  0.8   goose  :  0.8   goth  :  0.8   hand-feel  :  0.8   leading  :  0.8   map  :  0.8   noticeable  :  0.8   practice  :  0.8   produce  :  0.8   separates  :  0.8   shoe  :  0.8   slim-fitting  :  0.8   sweeping  :  0.8   tracy  :  0.8   trademarked  :  0.8   famous  :  0.7894736842105263   gauze  :  0.7894736842105263   angled  :  0.7857142857142857   apple  :  0.7857142857142857   sky  :  0.7857142857142857   destroyed  :  0.7777777777777778   fisherman  :  0.7777777777777778   smoothing  :  0.7777777777777778   skater  :  0.7772277227722773   min  :  0.7727272727272727   approximate  :  0.7722772277227723   functional  :  0.7715736040609137   expensive  :  0.7692307692307693   linning  :  0.7692307692307693   proud  :  0.7692307692307693   sleep  :  0.7692307692307693   taller  :  0.7692307692307693   age  :  0.7647058823529411   angeles  :  0.7608695652173914   these  :  0.7563909774436091   flatters  :  0.75555555555555

  flowly  :  0.5   freeprance  :  0.5   fresco  :  0.5   frilling  :  0.5   frills  :  0.5   full-figured  :  0.5   full-money-back  :  0.5   fuss  :  0.5   gal  :  0.5   gears  :  0.5   gesture  :  0.5   graded  :  0.5   grip  :  0.5   handcraft  :  0.5   heathers  :  0.5   hecho  :  0.5   hemp  :  0.5   hibiscus  :  0.5   high-waisted  :  0.5   hlbandage  :  0.5   iheartraves  :  0.5   imysty  :  0.5   inc  :  0.5   inelastic  :  0.5   infinite  :  0.5   infor  :  0.5   island  :  0.5   jazz  :  0.5   jessica  :  0.5   jet  :  0.5   kelly  :  0.5   laser-cut  :  0.5   lengh  :  0.5   light-weighted  :  0.5   linings  :  0.5   loungers  :  0.5   ma  :  0.5   mandala  :  0.5   materil  :  0.5   menswear  :  0.5   mock-wrap  :  0.5   multi-pocket  :  0.5   navel  :  0.5   no-wrinkle  :  0.5   non-itch  :  0.5   non-see-through  :  0.5   odering  :  0.5   omni-shade  :  0.5   orientation  :  0.5   outlines  :  0.5   p  :  0.5   paint  :  0.5   pane  :  0.5   panelled  :  0.5   panties  :

   weeding  :  0.26666666666666666   relaxation  :  0.2647058823529412   logos  :  0.2631578947368421   skirt  :  0.2583166332665331   ponte  :  0.25280898876404495   anti-tearing  :  0.25   availability  :  0.25   bottles  :  0.25   clouds  :  0.25   coins  :  0.25   damage  :  0.25   delicates  :  0.25   dream  :  0.25   earthy  :  0.25   encased  :  0.25   endeavors  :  0.25   excursions  :  0.25   flatterring  :  0.25   fullness  :  0.25   gathers  :  0.25   guess  :  0.25   gunmetal  :  0.25   gustav  :  0.25   hopping  :  0.25   imperial  :  0.25   industry  :  0.25   leotard  :  0.25   lifestyles  :  0.25   mountaineering  :  0.25   non-functional  :  0.25   non-see  :  0.25   opulent  :  0.25   ounce  :  0.25   para  :  0.25   peekaboo  :  0.25   permanent  :  0.25   pilates  :  0.25   piped  :  0.25   pleather  :  0.25   popping  :  0.25   push-up  :  0.25   queries  :  0.25   royalblue  :  0.25   scratching  :  0.25   shoots  :  0.25   soil  :  0.25   stash  :  0.25   stetch 

## 02. tf idf 계산

In [4]:
#path 에 있는 모든 id.json 파일을 가지고, tf를 구하여 "ID" : tf   꼴의 딕셔너리 리턴
#path 여러개로 재정의
def computeAllTF(paths):
    count = 0
    tfDict = {} 
    for path in paths:
        print("패스 돌았음")
        for item in path.iterdir():
            if(item.is_dir()): #안보이는 디렉토리가 있다.. 주의 ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
                continue
            itemJson = json.loads(item.read_text(), encoding='utf-8')
            features = itemJson["features"]
            ID = itemJson["id"]    
            tf = computeTF(bow, features)
            tfDict[ID] = tf
    return tfDict

# ---------------메인1(tf 전부 계산)---------------
tfDict = {} #tf 들을 저장
tfDict = computeAllTF(paths)
# ______메인2(idf 계산)___________
idf = computeIDF(tfDict)
#displayTupList(sortDict(idf), -1, withVal=True)

# 저장! (또 계산하지말고 불러쓰자)
#json.dump(idf, open(f'/home/jovyan/work/분석/idf' + '.json', 'w+')) 
#print(len(idf))
#json.dump(tfDict, open(f'/home/jovyan/work/분석/TFs' + '.json', 'w+')) 
#print(len(tfDict))

# 03. tf * idf * R/NR 

In [36]:
def computeIDFdivTF_R(tfBow, idfs,rnr):
    idftf = {} 
    for word, val in tfBow.items():
        #if val == 0:
        #    idftf[word] = 0
        if idfs[word] == 0:
            idftf[word] = 0
        else:
            #idftf[word] = idfs[word] / val * rnr[word] #이거 바꿔보자 _------------------------------------__ㅉ#%#$%@%@#@^@$
            idftf[word] = val / idfs[word] * rnr[word] #이거 바꿔보자 _------------------------------------__ㅉ#%#$%@%@#@^@$
    return idftf
def computeTFIDF_R(tfBow, idfs,rnr): 
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val * idfs[word] * rnr[word]
    return tfidf

tfidfDict = {} #  {ID : tf- * idf}
idftfDict = {} # {ID : idf / tf}
tfidfDict_R = {} # {ID : tf * idf * rNr}
idftfDict_R = {} # {ID : idf / tf * rNr}

counter = 0
for item in path1.iterdir():
    if(item.is_dir()): #안보이는 디렉토리 주의
        continue
    itemJson = json.loads(item.read_text(), encoding='utf-8')
    features = itemJson["features"]
    ID = itemJson["id"]
    #-------------------------(일반 : tf * idf)-----------------------
    tfidf = computeTFIDF(tfDict[ID], idf)
    tfidfDict[ID] = tfidf
    #-------------------------(시도1 : tf / idf)--------------------
    idftf = computeIDFdivTF(tfDict[ID], idf)
    idftfDict[ID] = idftf
    #------------------------(시도2 : tf * idf * RNR)------
    tfidf_R = computeTFIDF_R(tfDict[ID], idf, rnrDict)
    tfidfDict_R[ID] = tfidf_R
    #-----------------------(시도3: tf / idf * RNR)-------
    idftf_R = computeIDFdivTF_R(tfDict[ID], idf, rnrDict)
    idftfDict_R[ID] = idftf_R
    #-----------------------(출력)-----------------------------------------
    print("<features>")
    print(features)
    #print("< TF >")
    #displayTupList(sortDict(tfDict[ID]), -1)
    print(" < TF x IDF > ")
    displayTupList(sortDict(tfidf), 5)
    print(" < TF X IDF X RNR >")
    displayTupList(sortDict(tfidf_R), 5)
    print(" < TF / IDF >")
    displayTupList(sortDict(idftf), 5)
    print(" < TF / IDF * RNR >")
    displayTupList(sortDict(idftf_R), 5)
    counter += 1
    if counter == 10:
        break

<features>
['100% Polyester', 'Imported', 'Dry Clean Only', 'Empire-waist gown featuring ruched bust and concealed side zipper']
 < TF x IDF > 
empire-waist    ruched    gown    concealed    featuring    
 < TF X IDF X RNR >
empire-waist    gown    bust    ruched    concealed    
 < TF / IDF >
empire-waist    ruched    gown    concealed    featuring    
 < TF / IDF * RNR >
empire-waist    bust    and    gown    polyester    
<features>
['Material :100%Polyester', 'Feature:Long Sleeve,1/4 Zipper,Long Length,Hooded,Pocket, Color Block /Sherpa/Fleece/Pullover/Sweater/Sweatshirts/Outwear/Jacket/Coat', 'Suitable for daily wear,work,school,dates,sports,vacation,street wear. Pair with Jeans or Legging & Heels/Booties', 'Season: Spring, Autumn and Winter.', 'Wash Recommended With Cold Water/Do Not Bleach/Hang Or Line Dry']
 < TF x IDF > 
booties    sherpa    legging    outwear    dates    
 < TF X IDF X RNR >
sherpa    hooded    pullover    sleeve    sweatshirts    
 < TF / IDF >
booties    sh

### category 고치기

In [ ]:
{'정상1': 17911, '정상2': 0, '정상3': 0, '드레스': 8570, '바지': 4066, '새드레스': 434, '머냐': 337}

path = Path('/home/jovyan/work/크롤링/relFeatures')
path2 = Path('/home/jovyan/work/크롤링/nonRelFeatures')
paths = [path, path2]

docs = []
emptyList = []
pathCounter = 0
exCounter = 0
dic = {'정상1' : 0, '정상2':0, '정상3':0, '드레스': 0, '바지' :0, '새드레스':0, '머냐':0}
for path in paths:
    pathCounter += 1 
    for item in path.iterdir():
        if(item.is_dir()): 
            continue
        itemJson = json.loads(item.read_text(), encoding='utf-8')
        try:
            cates = itemJson["fullCategory"]
            ID = itemJson["id"]
        except:
            #B01AA6PPXU #미쳤나..;;이게 왜 섞였지.. 나중에 삭제나 수정 
            continue
        category = itemJson["category"]
        start = ""
        for cate in cates:
            start += cate
        if category == "sweater" or category == "hoodies&sweatshirts" or category == "skirt": 
            #정상적으로 카테고리 지정 됨
            dic["정상1"] += 1
            continue
        elif start.lower().find("pants") != -1:
            itemJson["category"] = "pants"
            json.dump(itemJson, open(f'/home/jovyan/work/크롤링/nonRelFeatures/' + ID + '.json', 'w+')) 
            #바지주인 찾음
            dic["바지"] += 1
        elif start.lower().find("dress") != -1:
            itemJson["category"] = "dress"
            dic["드레스"] += 1
            json.dump(itemJson, open(f'/home/jovyan/work/크롤링/relFeatures/' + ID + '.json', 'w+')) 
            #드레스주인찾음
        else:
            if start.strip() == "":
                emptyList.append(ID)
                #나중에 모아서 한번에 지우자..
            elif pathCounter == 1:
                itemJson["category"] = "dresse"
                json.dump(itemJson, open(f'/home/jovyan/work/크롤링/relFeatures/' + ID + '.json', 'w+'))
                dic["새드레스"] += 1
            elif pathCounter == 2: #출처를 알 수 없는 녀석들은.. nonRel이라고 만들어버림
                itemJson["category"] = "nonRel"
                dic["머냐"] += 1
                print(ID, " , ", start)
                #json.dump(itemJson, open(f'/home/jovyan/work/크롤링/nonRelFeatures/' + ID + '.json', 'w+')) 
print(dic)
